<a href="https://colab.research.google.com/github/YDHui/Tensorflow_Study/blob/master/Logic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
max_num_checkpoint = 10
num_classes = 2
batch_size = 512
num_epochs = 10
initial_learning_rate = 0.001
learning_rate_decay_factor = 0.95
num_epochs_per_decay = 1
is_training = False
fine_tuning = False
online_test = True
allow_soft_placement = True
log_device_placement = False
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", reshape=True, one_hot=False)
data={}
data['train/image'] = mnist.train.images
data['train/label'] = mnist.train.labels
data['test/image'] = mnist.test.images
data['test/label'] = mnist.test.labels
index_list_train = []
for sample_index in range(data['train/label'].shape[0]):
    label = data['train/label'][sample_index]
    if label == 1 or label == 0:
        index_list_train.append(sample_index)
index_list_test = []
for sample_index in range(data['test/label'].shape[0]):
    label = data['test/label'][sample_index]
    if label == 1 or label == 0:
        index_list_test.append(sample_index)
data['train/image'] = mnist.train.images[index_list_train]
data['train/label'] = mnist.train.labels[index_list_train]
data['test/image'] = mnist.test.images[index_list_test]
data['test/label'] = mnist.test.labels[index_list_test]
dimensionality_train = data['train/image'].shape
num_train_samples = dimensionality_train[0]
num_features = dimensionality_train[1]

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [8]:
graph = tf.Graph()
with graph.as_default():
    global_step = tf.Variable(0, name="global_step", trainable=False)
    decay_steps = int(num_train_samples / batch_size * num_epochs_per_decay)
    learning_rate = tf.train.exponential_decay(initial_learning_rate,
                                               global_step,
                                               decay_steps,
                                               learning_rate_decay_factor,
                                               staircase=True,
                                               name='exponential_decay_learning_rate')
    image_place = tf.placeholder(tf.float32, shape=([None, num_features]), name='image')
    label_place = tf.placeholder(tf.int32, shape=([None,]), name='gt')
    label_one_hot = tf.one_hot(label_place, depth=num_classes, axis=-1)
    dropout_param = tf.placeholder(tf.float32)
    logits = tf.contrib.layers.fully_connected(inputs=image_place, num_outputs = num_classes, scope='fc')
    with tf.name_scope('loss'):
        loss_tensor = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label_one_hot))
        prediction_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(label_one_hot, 1))
        accuracy = tf.reduce_mean(tf.cast(prediction_correct, tf.float32))
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    with tf.name_scope('train_op'):
        gradients_and_variables = optimizer.compute_gradients(loss_tensor)
        train_op = optimizer.apply_gradients(gradients_and_variables, global_step=global_step)
    session_conf = tf.ConfigProto(
        allow_soft_placement=allow_soft_placement,
        log_device_placement=log_device_placement)
    sess = tf.Session(graph=graph, config=session_conf)
    with sess.as_default():

        # The saver op.
        saver = tf.train.Saver()

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        # The prefix for checkpoint files
        checkpoint_prefix = 'model'

        # If fie-tuning flag in 'True' the model will be restored.
        if fine_tuning:
            saver.restore(sess, os.path.join(checkpoint_path, checkpoint_prefix))
            print("Model restored for fine-tuning...")
        test_accuracy = 0
        for epoch in range(num_epochs):
            total_batch_training = int(data['train/image'].shape[0] / batch_size)

            # go through the batches
            for batch_num in range(total_batch_training):
                #################################################
                ########## Get the training batches #############
                #################################################

                start_idx = batch_num * batch_size
                end_idx = (batch_num + 1) * batch_size

                # Fit training using batch data
                train_batch_data, train_batch_label = data['train/image'][start_idx:end_idx], data['train/label'][
                                                                                             start_idx:end_idx]

                ########################################
                ########## Run the session #############
                ########################################

                # Run optimization op (backprop) and Calculate batch loss and accuracy
                # When the tensor tensors['global_step'] is evaluated, it will be incremented by one.
                batch_loss, _, training_step = sess.run(
                    [loss_tensor, train_op,
                     global_step],
                    feed_dict={image_place: train_batch_data,
                               label_place: train_batch_label,
                               dropout_param: 0.5})
            print("Epoch " + str(epoch + 1) + ", Training Loss= " + \
                  "{:.5f}".format(batch_loss))

        ############################################################################
        ########## Run the session for pur evaluation on the test data #############
        ############################################################################

        # Evaluation of the model
        test_accuracy = 100 * sess.run(accuracy, feed_dict={
            image_place: data['test/image'],
            label_place: data['test/label'],
            dropout_param: 1.})

        print("Final Test Accuracy is %% %.2f" % test_accuracy)                

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch 1, Training Loss= 0.27870
Epoch 2, Training Loss= 0.12115
Epoch 3, Training Loss= 0.07586
Epoch 4, Training Loss= 0.05603
Epoch 5, Training Loss= 0.04468
Epoch 6, Training Loss= 0.03721
Epoch 7, Training Loss= 0.03190
Epoch 8, Training Loss= 0.02794
Epoch 9, Training Loss= 0.02488
Epoch 10, Training Loss= 0.02246
Final Test Accuracy is % 99.95
